## Generation of the GT for Training

In [ ]:
import math
import sys
import time
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import spatial

from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map

from sphere import Sphere
from visualize import Visualize
from semantic_classes import SemanticClasses
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# NUSCENES LOADER
# ----------------------------------------------------------------

export_ds = '...'
sem_clouds_filename = f"{export_ds}/teaser.npy"

print(f"Loading from sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename)
# sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

In [ ]:
# KITTI LOADER
# ----------------------------------------------------------------

export_ds = '...'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
    
# Remove intensity channel
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

In [ ]:
# WAYMO LOADER
# ----------------------------------------------------------------

export_ds = '...'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
    
# Remove intensity channel
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

In [ ]:
# POSS LOADER
# ----------------------------------------------------------------

export_ds = '...'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))    
    
# Remove intensity channel
print(f"Shape of sem clouds before intensity removal is {sem_cloud_features.shape}")
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

In [ ]:
# USL LOADER
# ----------------------------------------------------------------

export_ds = '...'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
    
# Remove intensity channel
print(f"Shape of sem clouds before is {sem_cloud_features.shape}")
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

In [ ]:
# PC Urban LOADER
# ----------------------------------------------------------------

export_ds = '...'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
    
# Remove intensity channel
print(f"Shape of sem clouds before is {sem_cloud_features.shape}")
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

In [ ]:
# A2D2 LOADER
# ----------------------------------------------------------------

export_ds = '...'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
    
# Remove intensity channel
print(f"Shape of sem clouds before is {sem_cloud_features.shape}")
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

In [ ]:
# CUSTOM LOADER
# ----------------------------------------------------------------

export_ds = '...'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
    
# Remove intensity channel
print(f"Shape of sem clouds before is {sem_cloud_features.shape}")
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

In [ ]:
n_clouds = sem_cloud_features.shape[0]
idx_sem = 1
classes = [None] * n_clouds
for i in tqdm(range(0, n_clouds)):
    sem_cloud = np.reshape(sem_cloud_features[i,:,:,:], (2, -1)).T    
    classes[i] = sem_cloud[:, idx_sem]
classes = np.array(classes[:])
print(f"shape of classes is {classes.shape}")
unique_classes = np.unique(classes)
n_unique_classes = len(unique_classes)
print(f"we have {n_unique_classes} classes in the dataset. That is:")
print(unique_classes)

## Iterate over the features and generate the GT

In [ ]:
n_features = sem_cloud_features.shape[0]
n_rows = sem_cloud_features.shape[2]
n_cols = sem_cloud_features.shape[3]
idx_range = 0
idx_sem = 1
max_range = 50

print(f'Iterating for {n_features} features and data of size: ({n_rows}/{n_cols})')
for feature in tqdm(range(n_features)):
    for i in range(n_rows):
        for j in range(n_cols):            
            cur_class = int(sem_cloud_features[feature, idx_sem, i, j])
#             mapped_class = SemanticClasses.map_nuscenes_label_old(cur_class)
            mapped_class = SemanticClasses.map_nuscenes_label(cur_class)
            # mapped_class = SemanticClasses.map_waymo_to_nuscenes_label(cur_class)
            # mapped_class = SemanticClasses.map_kitti_to_nuscenes_label(cur_class)
#             mapped_class = SemanticClasses.map_poss_to_nuscenes_label(cur_class)
#             mapped_class = SemanticClasses.map_usl_to_nuscenes_label(cur_class)
            # mapped_class = SemanticClasses.map_a2d2_to_nuscenes_label(cur_class)
            # mapped_class = SemanticClasses.map_pc_urban_to_nuscenes_label(cur_class)
            if (sem_cloud_features[feature, idx_range, i, j] <= max_range):
                sem_cloud_features[feature, idx_sem, i, j] = mapped_class
            else:
                sem_cloud_features[feature, idx_sem, i, j] = 0

In [ ]:
filename = 'sem_teaser'
sem_clouds_gt_filename = f'{export_ds}/{filename}.npy'
np.save(sem_clouds_gt_filename, sem_cloud_features)
print(f"Wrote features to {sem_clouds_gt_filename}")

# Create a tiny dataset
# n_process = 400
# sem_clouds_gt_tiny_filename = f'{export_ds}/{filename}_tiny.npy'
# np.save(sem_clouds_gt_tiny_filename, sem_cloud_features[0:n_process, :, :, :])
# print(f"Wrote tiny features to {sem_clouds_gt_tiny_filename}")

In [ ]:
# Remove a few samples for validation
n_clouds = sem_cloud_features.shape[0]
print(f'we have in total {n_clouds}')
n_train = n_clouds - 500
n_val = n_clouds - n_train
print(f'we will keep {n_train} for training and {n_val} for validation.')

print(f'init shape is {sem_cloud_features.shape}')
sem_train = sem_cloud_features[:n_train, :, :, :]
sem_val = sem_cloud_features[n_train:, :, :, :]
print(f'training shape is {sem_train.shape}, validation shape is {sem_val.shape}')

np.save(f'{export_ds}/sem_clouds3.npy', sem_train)
np.save(f'{export_ds}/sem_clouds_val_200.npy', sem_val)

print(f'Wrote features to {export_ds}/sem_clouds3.npy')
print(f'Wrote tiny features to {export_ds}/sem_clouds_val_200.npy')

In [ ]:
sem_val_tiny = sem_val[:500, :, :, :]
np.save(f'{export_ds}/sem_clouds_val_tiny.npy', sem_val_tiny)